In [1]:
import re
import requests
import time
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
options =  Options()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

In [3]:
categories = [
    'https://www.amazon.com/gp/bestsellers/baby-products',
    'https://www.konga.com/category/phones-tablets-5294',
    'https://www.konga.com/category/home-kitchen-602'
]

In [4]:
categories

['https://www.amazon.com/gp/bestsellers/baby-products',
 'https://www.konga.com/category/phones-tablets-5294',
 'https://www.konga.com/category/home-kitchen-602']

In [36]:
first_url = categories[1]

In [37]:
driver.get(first_url)

In [38]:
body_el = driver.find_element(by=By.CSS_SELECTOR, value='body')
html_str = body_el.get_attribute('innerHTML')

In [39]:
html_obj = HTML(html=html_str)
# print(html_obj.links)

In [40]:
page_links = [f'https://www.konga.com{x}' for x in html_obj.links if x.startswith('/')]

In [51]:
# page_links

In [42]:
def scrape_product_page(url, title_lookup = "._24849_2Ymhg", price_lookup = "._678e4_e6nqh"):
    driver.get(url)
    time.sleep(1.2)
    body_el = driver.find_element(by=By.CSS_SELECTOR, value='body')
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [43]:
# my_regex_pattern = r"https://www.konga.com/product/(?P<slug>[\w-]+)-(?P<product_id>[\w-]+)"
# my_url = 'https://www.konga.com/product/hisense-12-5kg-7-5kg-wash-5kg-spin-twin-tub-washing-machine-wspa753-5668850'

In [44]:
# regex = re.compile(my_regex_pattern)

In [45]:
# my_match = regex.match(my_url)
# print(my_match)

In [46]:
# my_match['product_id']

In [47]:
my_regex_pattern = r"https://www.konga.com/product/(?P<slug>[\w-]+)-(?P<product_id>[\w-]+)"

def extract_product_id_from_url(url):
    product_id = None
    regex = re.compile(my_regex_pattern)
    match = regex.match(url)
    if match != None:
        try:
            product_id = match['product_id']
        except:
            pass
    return product_id
            

In [48]:
def clean_page_links(page_links=[]):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url":url, "product_id": product_id})
    return final_page_links

cleaned_links = clean_page_links(page_links)

In [55]:
len(page_links) #== len(cleaned_links)

47

In [56]:
len(cleaned_links)

5

In [59]:
def perform_scrape(cleaned_items = []):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {
            'url':link,
            'product_id':product_id,
            'title':title,
            'price':price
        }  
        data_extracted.append(product_data)
    return data_extracted

In [61]:
extracted_data = perform_scrape(cleaned_items = cleaned_links)

https://www.konga.com/product/infinix-smart-hd-x612-2021-blue-32gb-2gb-infinix-headset-5066095 Infinix Smart Hd-x612- 2021 - Blue 32gb /2gb + Infinix Headset ₦46,300
https://www.konga.com/product/tecno-phantom-x-ac8-256-8gb-branded-bag-monet-summer-6-7inch-4700mah-50mp-13mp-8m-super-amoled-5314645 Tecno Phantom X (ac8) 256+8gb + Branded Bag - Monet Summer- 6.7inch -4700mah - 50mp+13mp+8m Super Amoled ₦240,000
https://www.konga.com/product/asus-x415ja-bv192t-14-hd-intel-i-core-o-i3-1005g1-4gb-ram-1tb-hdd-win10-transparent-silver-5497503 Asus X415ja-bv192t - 14" Hd -Intel® Core™ I3-1005g1 - 4GB RAM - 1TB HDD - Win10- Transparent Silver ₦220,000
https://www.konga.com/product/infinix-32-hd-smart-android-television-5102735 Infinix 32' HD Smart Android Television ₦87,600
https://www.konga.com/product/tecno-camon-18-premier-ch9-67-256gb-8gb-dual-sim-polar-night-4750mah-5480394 Tecno Camon 18 Premier -CH9- 6'7" - 256gb - 8gb - Dual Sim - Polar Night - 4750mah ₦180,200


In [62]:
print(extracted_data)

[{'url': 'https://www.konga.com/product/infinix-smart-hd-x612-2021-blue-32gb-2gb-infinix-headset-5066095', 'product_id': '5066095', 'title': 'Infinix Smart Hd-x612- 2021 - Blue 32gb /2gb + Infinix Headset', 'price': '₦46,300'}, {'url': 'https://www.konga.com/product/tecno-phantom-x-ac8-256-8gb-branded-bag-monet-summer-6-7inch-4700mah-50mp-13mp-8m-super-amoled-5314645', 'product_id': '5314645', 'title': 'Tecno Phantom X (ac8) 256+8gb + Branded Bag - Monet Summer- 6.7inch -4700mah - 50mp+13mp+8m Super Amoled', 'price': '₦240,000'}, {'url': 'https://www.konga.com/product/asus-x415ja-bv192t-14-hd-intel-i-core-o-i3-1005g1-4gb-ram-1tb-hdd-win10-transparent-silver-5497503', 'product_id': '5497503', 'title': 'Asus X415ja-bv192t - 14" Hd -Intel® Core™ I3-1005g1 - 4GB RAM - 1TB HDD - Win10- Transparent Silver', 'price': '₦220,000'}, {'url': 'https://www.konga.com/product/infinix-32-hd-smart-android-television-5102735', 'product_id': '5102735', 'title': "Infinix 32' HD Smart Android Television", 